In [1]:
from itertools import chain
from multiprocess import pool

In [2]:
with open('../data/2024/day6.txt') as f:
   data = f.read()

In [3]:
grid = list([list(line) for line in data.splitlines()])
grid_extents = {'x0': 0, 'x1': len(grid[0]) - 1, 'y0': 0, 'y1': len(grid) - 1}

In [4]:
def find_in_grid(value):
    return list(chain(*[[(y,index)
        for index, element in enumerate(row)
        if element == value]
        for y,row in enumerate(grid)]))

def vector_rotate_90(v):
    return v[1], -v[0]

def vector_add(v1, v2):
   return tuple([v1[i] + v2[i] for i in range(len(v1))])

In [5]:
obstacles = set(find_in_grid('#'))

In [6]:
# Part 1
visited = set()
guard_at = find_in_grid('^')[0]
guard_dir = (-1,0) # north

while True:
    # Keep track of our past steps (plus starting location)
    visited.add(guard_at)

    # Look ahead
    next_step = vector_add(guard_at, guard_dir)

    # If the next step is out of bounds, we're done
    if not (
        grid_extents['x0'] <= next_step[0] <= grid_extents['x1']
        and grid_extents['y0'] <= next_step[1] <= grid_extents['y1']
    ): break
    # If we hit an obstacle, rotate clockwise 90º
    elif next_step in obstacles: guard_dir = vector_rotate_90(guard_dir)
    # Otherwise move
    else: guard_at = next_step

print("Part 1: {}".format(len(visited)))

Part 1: 4656


In [7]:
# Part 2
# Only try positions we previously walked in part 1
guard_start = find_in_grid('^')[0]
prospective_locations = set(visited).difference([guard_start])

def guard_walk(new_obstacle):
    # Reset our state every attempt
    visited = set()
    guard_at = guard_start
    guard_dir = (-1,0) # north

    # Clone the original obstacles and add our new one
    prospective_obstacles = set(obstacles)
    prospective_obstacles.add(new_obstacle)

    while True:
        # If we repeat a location + heading we found a cycle
        if (*guard_at, guard_dir) in visited:
            return new_obstacle

        # Keep track of our past steps and headings
        visited.add((*guard_at, guard_dir))

        # Look ahead
        next_step = vector_add(guard_at, guard_dir)

        # If the next step is out of bounds, abort this attempt
        if not (
            grid_extents['x0'] <= next_step[0] <= grid_extents['x1']
            and grid_extents['y0'] <= next_step[1] <= grid_extents['y1']
        ): break

        # If we hit an obstacle, rotate clockwise 90º
        elif next_step in prospective_obstacles: guard_dir = vector_rotate_90(guard_dir)
        # Otherwise move
        else: guard_at = next_step

with pool.Pool() as p:
    results = p.map(guard_walk, prospective_locations)

print("Part 2:", len(list(filter(None, results))))

Part 2: 1575
